In [ ]:
import numpy as np
import pandas as pd
# 0=Neurons, 4=Astrocytes,5=Oligodendrocytes,11=Microglia,13=Endothelial #

In [3]:
clustann = pd.read_csv('/Users/brianbrogan/Desktop/KI24/ClusterImgGen2024/STERSEQ/output/01. Co-expression network/1996-081_GFM_SS200000954BR_A2_bin100_tissue_cleaned/Cluster_annotation.csv')
print(clustann.shape)
clustann.head()

(21441, 4)


,features,x,y,cluster
0,WASH7P,0.058166,8.574180,0
1,WASH9P,10.845608,10.232767,2
2,MTND1P23,-0.641904,10.953246,0
3,MTND2P28,-4.345322,8.884488,0
4,MTCO1P12,-0.549818,10.905572,0


In [26]:
duplicates = clustann[clustann.duplicated(subset=['features'], keep=False)]
print(duplicates.shape)
duplicates.head()

(0, 4)


,features,x,y,cluster


In [27]:
print(np.unique(clustann['cluster']))

[-1  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22
 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46
 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70
 71 72 73 74]


In [38]:
clust_0 = clustann.loc[clustann['cluster']==4, :]
print(clust_0.shape)
clust_0.head()


(54, 4)


,features,x,y,cluster
35,MXRA8,18.692465,10.252989,4
236,ESPNP,18.730371,10.231768,4
253,TAS1R2,18.150196,10.561714,4
1350,S100A2,18.511414,10.324617,4
1431,RAB25,18.462917,10.554472,4


In [39]:
clust_0_genes = np.unique(clust_0['features'])
print(clust_0_genes.shape)

(54,)


In [40]:
clust_0_gene_set = set(clust_0_genes)

In [41]:
sterseq_df = pd.read_csv('/Users/brianbrogan/Desktop/KI24/ClusterImgGen2024/STERSEQ/input/1996-081_GFM_SS200000954BR_A2_tissue_cleaned_cortex_crop.csv')

In [42]:
print(sterseq_df.shape)
sterseq_df.head()

(9084839, 8)


,Unnamed: 0,geneID,x,y,MIDCount,ExonCount,IntronCount,bin1_ID
0,1,WASH7P,4209,8106,1,0,1,159121569.0
1,2,WASH7P,8313,6307,1,0,1,123807705.0
2,3,WASH7P,6394,8031,1,0,1,157651354.0
3,4,WASH7P,7620,10189,3,0,3,200018436.0
4,5,WASH7P,10163,8238,1,0,1,161718947.0


In [43]:
clust_0_sterseq_df = sterseq_df[sterseq_df['geneID'].isin(clust_0_gene_set)]

In [44]:
print(clust_0_sterseq_df.shape)
clust_0_sterseq_df.head()

(5682, 8)


,Unnamed: 0,geneID,x,y,MIDCount,ExonCount,IntronCount,bin1_ID
35433,35434,MXRA8,9143,6520,1,0,1,127990151.0
35434,35435,MXRA8,5622,9367,1,1,0,183878934.0
35435,35436,MXRA8,9205,9114,1,1,0,178915621.0
35436,35437,MXRA8,7778,7028,1,1,0,137961842.0
35437,35438,MXRA8,5364,11161,1,1,0,219098484.0


In [45]:
indicators = clust_0_sterseq_df['geneID'].value_counts()



In [46]:
# Display the top N most common genes (e.g., top 10)
top_n_common_genes = indicators.head(25)
print(top_n_common_genes)
#print(indicators[:20])

geneID
TCF12          754
STAT3          654
NOS1           311
ZSWIM7         302
DHCR7          279
ZNF44          253
AATF           251
KATNA1         229
NFKBIB         208
ANO1           183
CFAP20DC-DT    170
MCAM           150
TPTE2P6        123
PLS1           112
ENOSF1         106
CCNJL          102
PIWIL2          99
MXRA8           97
SCN10A          86
XAF1            85
ZNF440          85
MCM9            82
MOCOS           81
MUC16           81
TPTE2P5         66
Name: count, dtype: int64


In [35]:
gene_cluster_counts = sterseq_df.groupby(['cluster', 'geneID']).size().reset_index(name='count')
gene_cluster_counts.head()

KeyError: 'cluster'

In [ ]:
def find_gene_set(df, clust_id):
    clust = df.loc[df['cluster']==clust_id, :]
    clust_genes_set = set(np.unique(clust['features']))
    return clust_genes_set

def create_cluster_df(origin, clust_id):
    gene_set = find_gene_set(origin, clust_id)
    cluster_df = origin[origin['geneID'].isin(gene_set)]
    return cluster_df

def find_n_indicators(cluster_df, minimum_count):
    indicators = cluster_df['geneID'].value_counts()
    indicators = indicators.loc[indicators['count']>= minimum_count]
    return indicators